In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchinfo import summary
from tqdm import tqdm
from nn_zoo.datamodules import MNISTDataModule
from nn_zoo.models.components import *
import matplotlib.pyplot as plt

In [ ]:
dm = MNISTDataModule(
        data_dir="../../../data",
        dataset_params={
            "download": True,
            "transform": torchvision.transforms.Compose(
                [
                    torchvision.transforms.Resize((32, 32)),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.RandomRotation(30),
                ]
            ),
        },
        loader_params={
            "batch_size": 128
        },
    )

dm.setup()

train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()

In [ ]:
x, y = next(iter(val_loader))

plt.imshow(torchvision.utils.make_grid(x).permute(1, 2, 0))
plt.title("Images")
plt.axis("off")
plt.show()

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = DepthwiseSeparableConv2d(1, 6, 3, 1, 1)
        self.conv2 = DepthwiseSeparableConv2d(6, 16, 3, 1, 1)

        self.fc = nn.Sequential(
            nn.Linear(16 * 8 * 8, 10),
        )

        self.load_state_dict(torch.load("model2.pth"))

    def forward(self, x, lpips=False):
        x1 = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x2 = F.max_pool2d(F.relu(self.conv2(x1)), 2)

        x2 = x2.view(x2.size(0), -1)
        x = self.fc(x2)

        if lpips:
            return x, x1, x2

        return x
    
    def lpips(self, img1, img2):
        # Forward pass with lpips=True to get feature maps
        _, x1_features1, x2_features1 = self.forward(img1, lpips=True)
        _, x1_features2, x2_features2 = self.forward(img2, lpips=True)

        # Compute L2 distances between corresponding feature maps
        dist1 = F.mse_loss(x1_features1, x1_features2)
        dist2 = F.mse_loss(x2_features1, x2_features2)

        # Sum the distances to get the LPIPS score
        lpips_score = dist1 + dist2

        return lpips_score
    
model = Model()
model = model.to("mps")
print(summary(model, input_size=(1, 1, 32, 32), device="mps", depth=1))

In [ ]:
from ema_pytorch import EMA

ema = EMA(model, beta=0.999, update_after_step=2000, update_every=10)
optimizer = torch.optim.AdamW(model.parameters(), lr=4e-4)

In [ ]:
@torch.no_grad()
def eval(model, loader):
    model.eval()
    correct = 0
    total = 0
    loss = 0
    for x, y in loader:
        x, y = x.to("mps"), y.to("mps")
        y_hat = model(x)
        pred = y_hat.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()
        total += len(x)
        loss += F.cross_entropy(y_hat, y).item()
    return correct / total, loss / len(loader)

roll_loss = 0
for epoch in range(200):
    model.train()
    val_acc, val_loss = eval(model, val_loader)
    lpips = model.lpips(torch.ones(4, 1, 32, 32).to("mps"), torch.zeros(4, 1, 32, 32).to("mps")).item()
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}", ncols=150)
    for x, y in pbar:
        x, y = x.to("mps"), y.to("mps")
        optimizer.zero_grad()
        y_hat = model(x)
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()

        ema.update()

        roll_loss = 0.99 * roll_loss + 0.01 * loss.item()
        pbar.set_postfix_str(f"loss: {roll_loss:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.2%}, lpips: {lpips:.4f}")
    

In [ ]:
torch.save(model.state_dict(), "model2.pth")

In [ ]:
# plot the distribution of weights for each layer
plt.subplot()